## PURPOSE OF THE NOTEBOOK + TITLE OF THE PROJECT

In [2]:
import requests
from requests.adapters import HTTPAdapter
import json

import pandas as pd
from datetime import date
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
import time
import numpy as np
import re

In [2]:
links = ['https://www.ravelry.com/patterns/search#pc=medical&sort=name&view=captioned_thumbs&page=1','https://www.ravelry.com/patterns/search#pc=medical&sort=name&view=captioned_thumbs&page=2','https://www.ravelry.com/patterns/search#pc=medical&sort=name&view=captioned_thumbs&page=3','https://www.ravelry.com/patterns/search#pc=medical&sort=name&view=captioned_thumbs&page=4']

### Get the ID's of all the recipes in the medical knitting recipe category

In [22]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

url = links[0]
await page.goto(url)


await page.locator("#user_login").fill('xxxxx')
await page.locator("#user_password").fill('xxxxx')
await page.locator("#login_form > div.splash_page__submit > div.splash_page__submit__login > button").click()

time.sleep(5)

pattern_ids = []

# Scrape each of the four pages:
for link in links:
    await page.goto(link)
    time.sleep(5)
    html = await page.content()
    doc = BeautifulSoup(html)

    content_card = doc.find_all(class_= "pattern captioned_pattern captioned_search_result")

    pattern_id = []

    # Loop through each pattern on the page and get the information:
    for card in content_card:
        pattern = {}
        pattern['name'] = card.find('a').get('title')
        pattern['url'] = card.find('a').get('href')

        find_id = card.get('id')
        pattern['id'] = re.findall(r'\d+', find_id)[0]
        #find_id = card.get('id')
        #id = re.findall(r'\d+', find_id)[0]
        pattern_id.append(pattern)
    pattern_ids.append(pattern_id)

### And take the list of lists (with some dictionaries inside) created in the scrape and turn it into one flat list.

This allows me to turn it into a dataframe and then into a csv file.

In [33]:
flat_list = []
for ids in pattern_ids:
    for id in ids:
        flat_list.append(id)

id_df = pd.DataFrame(flat_list)
id_df

,name,url,id
0,"""Hey, at Least It Wasn't Your Leg"" Arm Sling b...",https://www.ravelry.com/patterns/library/hey-a...,80882
1,"""Kitty"" Eye Patch by Christine H. Wilkins",https://www.ravelry.com/patterns/library/kitty...,118789
2,Alternative ML Knitted Knockers by Makoshark,https://www.ravelry.com/patterns/library/alter...,951015
3,Amon ammeinnlegg by Anne Tvedt,https://www.ravelry.com/patterns/library/amon-...,807316
4,Amputee's Stump Covers by Suzetta Williams,https://www.ravelry.com/patterns/library/amput...,287776
...,...,...,...
165,Woolly Boob by Kathryn Cottam,https://www.ravelry.com/patterns/library/wooll...,639320
166,Worry Bear by Joanne Seiff,https://www.ravelry.com/patterns/library/worry...,908676
167,Wrapped in Lace PICC Line Cover by Kirsten Hol...,https://www.ravelry.com/patterns/library/wrapp...,749754
168,Wrist rest by Lynne Rowe,https://www.ravelry.com/patterns/library/wrist...,464528


In [35]:
id_df.to_csv("data/patterns.csv", index=False)

In [65]:
df = pd.read_csv("data/patterns.csv")

In [66]:
id_list = df.id.values.tolist()


In [67]:
url_list = df.url.values.tolist()
url_list

['https://www.ravelry.com/patterns/library/hey-at-least-it-wasnt-your-leg-arm-sling',
 'https://www.ravelry.com/patterns/library/kitty-eye-patch-2',
 'https://www.ravelry.com/patterns/library/alternative-ml-knitted-knockers',
 'https://www.ravelry.com/patterns/library/amon-ammeinnlegg',
 'https://www.ravelry.com/patterns/library/amputees-stump-covers',
 'https://www.ravelry.com/patterns/library/ankle-wrap-ice-packs',
 'https://www.ravelry.com/patterns/library/arm-cast-covers',
 'https://www.ravelry.com/patterns/library/arm-cast-sleeve',
 'https://www.ravelry.com/patterns/library/arm-sling',
 'https://www.ravelry.com/patterns/library/bandages-for-leprosy-patients',
 'https://www.ravelry.com/patterns/library/boo-boo-buddy',
 'https://www.ravelry.com/patterns/library/boo-boo-buddies',
 'https://www.ravelry.com/patterns/library/boo-boo-buddies-2',
 'https://www.ravelry.com/patterns/library/boo-boo-bunny-knit-pouch',
 'https://www.ravelry.com/patterns/library/boob',
 'https://www.ravelry.co

In [68]:
project_numbers = []

for url in url_list:
    await page.goto(url)

    time.sleep(0.5)

    project = {}

    html = await page.content()
    doc = BeautifulSoup(html)

    type = doc.find_all(class_='value')
    type = type[1].text
    type = re.sub(r'\n', '', type)

    project['craft_type'] = type
    
    projects = doc.find(class_= "pattern_people summary_box_people").text
    # replace \n\n\n with nothing
    projects = re.sub(r'\n\n\n', '', projects)
    project['project_numbers'] = re.sub(r' projec\D*,\nin \d* queues\n\n', '', projects)
    
    project_numbers.append(project)
await browser.close()

In [72]:
df = pd.DataFrame(project_numbers)
df.to_csv("data/patterns_info.csv", index=False)

Use the Ravelry API to get information about projects that uses each of the recipes in the medical knitting recipe category.

In [41]:
pattern_info = []
for id in col_list:
    api_key_id = 'xxxxxx'
    api_key_secret = 'xxxxxx'
    headers = {'Content-Type': 'application/json'}
    auth = (api_key_id, api_key_secret)
    newSession = requests.Session()
    link = f"https://api.ravelry.com/patterns/{id}/projects.json"
    response = newSession.post('https://api.ravelry.com/patterns/80882/projects.json',auth=auth,headers=headers,verify=False) 
    print(response.status_code)
    data = response.json()
    pattern_info.append(data)

/Users/laurabejderjensen/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ravelry.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/laurabejderjensen/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ravelry.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/laurabejderjensen/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ravelry.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26

200
200
200


/Users/laurabejderjensen/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ravelry.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/laurabejderjensen/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ravelry.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/laurabejderjensen/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ravelry.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26

200
200
200
200
200


/Users/laurabejderjensen/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ravelry.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/laurabejderjensen/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ravelry.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/laurabejderjensen/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ravelry.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26

...

/Users/laurabejderjensen/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ravelry.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/laurabejderjensen/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ravelry.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/laurabejderjensen/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ravelry.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26

200
200


/Users/laurabejderjensen/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ravelry.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/laurabejderjensen/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ravelry.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


200
200
200


/Users/laurabejderjensen/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ravelry.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/laurabejderjensen/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ravelry.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [44]:
pattern_info

[{'projects': [{'comments_count': 0,
    'completed': '2009/09/27',
    'craft_id': 2,
    'created_at': '2009/09/30 09:25:40 -0400',
    'favorites_count': 0,
    'id': 2844817,
    'made_for': '',
    'made_for_user_id': None,
    'name': 'Sling for Surgery',
    'pattern_id': 80882,
    'permalink': 'hey-at-least-it-wasnt-your-leg-arm-sling',
    'progress': 100,
    'project_status_changed': '2009/09/30 09:30:41 -0400',
    'project_status_id': 2,
    'rating': 4,
    'size': '',
    'started': '2009/08/01',
    'updated_at': '2009/11/02 14:54:07 -0500',
    'user_id': 148804,
    'links': {'self': {'href': 'https://www.ravelry.com/projects/Irishcorp/hey-at-least-it-wasnt-your-leg-arm-sling'}},
    'pattern_name': '"Hey, at Least It Wasn\'t Your Leg" Arm Sling',
    'craft_name': 'Knitting',
    'status_name': 'Finished',
    'tag_names': [],
    'first_photo': {'id': 8234306,
     'sort_order': 1,
     'x_offset': -36,
     'y_offset': -6,
     'square_url': 'https://images4-g.rav

In [ ]:

api_key_id = 'f5a6ec6d351e27877181d3b82cf96c4b'
api_key_secret = 'CC-gqxAxnUUfnZvAEImpCNMhX3zQoLzQYu1wIjC-'
headers = {'Content-Type': 'application/json'}
auth = (api_key_id, api_key_secret)
newSession = requests.Session()
response = newSession.post('https://api.ravelry.com/patterns/search.json?category=medical',auth=auth,headers=headers,verify=False) 
print(response.status_code)

In [ ]:
data = response.json()
data


In [19]:

api_key_id = 'f5a6ec6d351e27877181d3b82cf96c4b'
api_key_secret = 'CC-gqxAxnUUfnZvAEImpCNMhX3zQoLzQYu1wIjC-'
headers = {'Content-Type': 'application/json'}
auth = (api_key_id, api_key_secret)
newSession = requests.Session()
response = newSession.post('https://api.ravelry.com/patterns/503114/projects.json?page_size=5000',auth=auth,headers=headers,verify=False) 
print(response.status_code)
pattern_data = response.json()
len(pattern_data['projects'])


/Users/laurabejderjensen/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'api.ravelry.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


504


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [4]:
# CREATE A LOOP THAT LOOPS THROUGH ALL THE PATTERNS AND GET THE THE NUMBER OF PROJECTS FOR EACH PATTERN



{'projects': [{'comments_count': 0, 'completed': '2009/09/27', 'craft_id': 2, 'created_at': '2009/09/30 09:25:40 -0400', 'favorites_count': 0, 'id': 2844817, 'made_for': '', 'made_for_user_id': None, 'name': 'Sling for Surgery', 'pattern_id': 80882, 'permalink': 'hey-at-least-it-wasnt-your-leg-arm-sling', 'progress': 100, 'project_status_changed': '2009/09/30 09:30:41 -0400', 'project_status_id': 2, 'rating': 4, 'size': '', 'started': '2009/08/01', 'updated_at': '2009/11/02 14:54:07 -0500', 'user_id': 148804, 'links': {'self': {'href': 'https://www.ravelry.com/projects/Irishcorp/hey-at-least-it-wasnt-your-leg-arm-sling'}}, 'pattern_name': '"Hey, at Least It Wasn\'t Your Leg" Arm Sling', 'craft_name': 'Knitting', 'status_name': 'Finished', 'tag_names': [], 'first_photo': {'id': 8234306, 'sort_order': 1, 'x_offset': -36, 'y_offset': -6, 'square_url': 'https://images4-g.ravelrycache.com/uploads/Irishcorp/14682959/HPIM0230_square.JPG', 'medium_url': 'https://images4-f.ravelrycache.com/uplo